In [159]:
import requests
from bs4 import BeautifulSoup
import os
import json
import sys
sys.path.append("/Users/kuramochiosuke/.pyenv/versions/3.10.4/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages")
from requests_oauthlib import OAuth1Session
sys.path.append("/Users/kuramochiosuke/.pyenv/versions/3.10.4/lib/python3.10/site-packages")
from dotenv import find_dotenv, load_dotenv
import requests
import schedule
import time


# def job():
    # .envファイルを探して読み込み
env_file = find_dotenv()
load_dotenv(env_file)  

CONSUMER_KEY = os.environ.get('CONSUMER_KEY')
CONSUMER_SECRET = os.environ.get('CONSUMER_SECRET')
ACCESS_KEY = os.environ.get('ACCESS_KEY')
ACCESS_KEY_SECRET = os.environ.get('ACCESS_KEY_SECRET')

# Twitterの認証
twitter = OAuth1Session(CONSUMER_KEY, CONSUMER_SECRET, ACCESS_KEY, ACCESS_KEY_SECRET)
print(twitter)

#エンドポイント
url_text = 'https://api.twitter.com/1.1/statuses/update.json'
url_media = "https://upload.twitter.com/1.1/media/upload.json"



# # ここまでTwitter投稿の準備
# print("投稿準備完了")

# params_2 = {
#     'count':100,
#     'screen_name':'AhTp2vML1Ws95rO'
#     }
# res = twitter.get(url_user, params=params_2)
# abc = json.loads(res.text)
# print("過去100投稿分のデータ取得完了")



#スニだんのページの指定
URL = 'https://orenoraresne.com'
# リクエストヘッダの指定
headers = {"User-Agent": "hoge"}
response = requests.get(URL,  headers=headers)
r_text=response.text
soup = BeautifulSoup(r_text, 'html.parser')


print('レアスニのページの取得完了')



#     # 記事の取得 
#     soup_article=soup.find_all("article")[5]
#     new_elem=soup_article.find_all("img")[0]['src']
#     try:
#         with open('old_elem.txt') as f:
#             old_elem = f.read()
#     except:
#         old_elem=""
#     if new_elem == old_elem:
#         print("変化なし")
#         print("")
#     else:
#         with open("old_elem.txt","w") as f:
#             f.write(new_elem)   
#         soup_text=soup_article.find_all("h3")[0].find("a").text.replace("\n","").replace("\t","").replace("定価/","")
#         # 画像の取得
#         soup_img=soup_article.find_all("img")[0]['src']
#         # 詳細ページのリンクを取得
#         soup_url="https://snkrdunk.com/"+soup_article.find("a")['href']
#         # 画像の処理
#         response = requests.get(soup_img)
#         time.sleep(2)
#         image = response.content
#         files = {"media" : image}
#         req_media = twitter.post(url_media, files = files)
#         time.sleep(2)
#         media_id = json.loads(req_media.text)['media_id']
#         print('画像の取得完了')


#         print("ここから詳細ページ")
#         URL = soup_url
#         # リクエストヘッダの指定
#         headers = {"User-Agent": "hoge"}
#         response = requests.get(URL,  headers=headers)
#         time.sleep(2)
#         r_text=response.text
#         soup = BeautifulSoup(r_text, 'html.parser')
#         time.sleep(2)
        
        
#         if soup_text[:5]=="【リーク】":
#             soup_h1=soup.find_all("h1")[0].text.replace("抽選/定価/販売店舗まとめ","").replace("【リーク】","")
#             soup_p=soup.find_all("p",attrs={"class","page-title-en"})[0].text
#             params_1 = {'status':"リーク情報!!!\n\n{}\n\n{}\n\n情報が入り次第更新!!!".format(soup_h1,soup_p),'media_ids':[media_id]}
#             twitter.post(url_text, params = params_1)
#             time.sleep(2)
#             print("投稿完了しました")
#             print("")
            
            
#         elif soup_text[:5]=="【販売リン":
#             a_count=len(soup.find_all("div",attrs={"class","sneaker-release-shop-box"})[0].find_all("a"))
#             for i in range(a_count):
#                 soup_block=soup.find_all("div",attrs={"class","sneaker-release-shop-box"})[0].find_all("a")[i]
#                 soup_link=soup_block['href']
#                 soup_app_name=soup_block.find_all("div",attrs={"class","left-box"})[0].text
#                 soup_data=soup_block.find_all("div",attrs={"class","shop-right-box"})[0].text

#                 # ツイート(複数画像をアップロードする場合は["test1.jpg,test2.jpg"])みたいな感じ
#                 params_1 = {'status': "{}\n\n{}  {}\n{}\n".format(soup_text,soup_app_name,soup_data,soup_link),'media_ids':[media_id]}
#                 time.sleep(2)
#                 twitter.post(url_text, params = params_1)
#                 time.sleep(2)
#                 print("投稿完了しました")
#                 print("")
#         else:
#             print("これから実装")
#             print("")
            
                
# def main():
#     schedule.every(1).minutes.do(job)
#     while True:
#         schedule.run_pending()
#         time.sleep(1)

# if __name__ == '__main__':
#     main()

レアスニのページの取得完了


In [160]:
soup_article=soup.find_all("article",attrs={"class","post-list"})[0]

In [161]:
soup_img=soup_article.find_all("img")[0]['src']
soup_img

'https://orenoraresne.com/wp-content/uploads/2022/06/スクリーンショット-2022-06-15-21.14.34-485x278.jpg'

In [162]:
soup_name=soup_article.find_all("h1",attrs={"class","entry-title"})[0].text
soup_name

'【6月28日(火)販売予定】ユニオン × ナイキ コルテッツ SP【2カラー】'

In [163]:
soup_url=soup_article.find_all("a")[0]["href"]
soup_url

'https://orenoraresne.com/union-nike-cortez-sp-20220622'

In [164]:
# 画像の処理
response = requests.get(soup_img)
image = response.content
files = {"media" : image}
req_media = twitter.post(url_media, files = files)
media_id = json.loads(req_media.text)['media_id']
print('画像の取得完了')

画像の取得完了


In [165]:
print("ここから詳細ページ")
URL = soup_url
# リクエストヘッダの指定
headers = {"User-Agent": "hoge"}
response = requests.get(URL,  headers=headers)
r_text=response.text
soup = BeautifulSoup(r_text, 'html.parser')

ここから詳細ページ


In [166]:
soup_p=soup.find_all("p",attrs={"class","has-text-align-center"})
for i in range(len(soup_p)-1):
    soup_a=soup.find_all("p",attrs={"class","has-text-align-center"})[i].text
    if soup_a[:3]=="定価：":
        soup_purene=soup.find_all("p",attrs={"class","has-text-align-center"})[i].text.replace("俺的","")
soup_purene

'定価：15,400円(税込)プレ値(転売予想価格)：オフノワール→〜20,000円セサミ→〜17,000円'

In [167]:
params = {'status':"予想プレ値!!!\n\n{}\n\n{}".format(soup_name,soup_purene),'media_ids':[media_id]}
twitter.post(url_text, params = params)
print("投稿完了しました")
print("")

投稿完了しました



In [168]:
soup_h2=soup.find_all("h2",attrs={"class","has-text-align-left"})[0].text
soup_h2

'結果発表'

In [169]:
soup_p=soup.find_all("p",attrs={"class","has-text-align-center"})
for i in range(len(soup_p)):
    soup_a=soup.find_all("p",attrs={"class","has-text-align-center"})[i].text
    if soup_a[:2]=="結果":
        soup_result=soup.find_all("p",attrs={"class","has-text-align-center"})[i].text.replace("予想外れ","")
soup_result

'結果：転売価格：オフノワール→22,000〜25,000円セサミ→18,000〜20,000円'

In [170]:
params = {'status':"結果発表!!!\n\n{}\n\n{}".format(soup_name,soup_result),'media_ids':[media_id]}
twitter.post(url_text, params = params)
print("投稿完了しました")
print("")

投稿完了しました

